In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import datetime
import calendar

In [2]:
train_df = pd.read_csv('../data/ml_100k/train_data.csv')
train_df

,user_id,item_id,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712
...,...,...,...,...
79995,943,1067,2,875501756
79996,943,1074,4,888640250
79997,943,1188,3,888640250
79998,943,1228,3,888640275


In [3]:
item_df = pd.read_csv('../data/ml_100k/item_data.csv')
item_df

,id,title,release_date,video_release_date,url,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'], unit='s')
train_df

,user_id,item_id,rating,timestamp
0,1,1,5,1997-09-22 22:02:38
1,1,2,3,1997-10-15 05:26:11
2,1,3,4,1997-11-03 07:42:40
3,1,4,3,1997-10-15 05:25:19
4,1,5,3,1998-03-13 01:15:12
...,...,...,...,...
79995,943,1067,2,1997-09-29 02:55:56
79996,943,1074,4,1998-02-28 04:30:50
79997,943,1188,3,1998-02-28 04:30:50
79998,943,1228,3,1998-02-28 04:31:15


In [26]:
train_df['timestamp'].min()

Timestamp('1997-09-20 03:05:27')

In [27]:
train_df['timestamp'].max()

Timestamp('1998-04-22 23:10:38')

In [24]:
main_df = train_df.copy()
reference_date = main_df['timestamp'].min()
main_df['month_enc'] = main_df['timestamp'].apply(lambda x: (x.year - reference_date.year)*12 + (x.month - reference_date.month))
main_df.head()

,user_id,item_id,rating,timestamp,month_enc
0,1,1,5,1997-09-22 22:02:38,0
1,1,2,3,1997-10-15 05:26:11,1
2,1,3,4,1997-11-03 07:42:40,2
3,1,4,3,1997-10-15 05:25:19,1
4,1,5,3,1998-03-13 01:15:12,6


In [25]:
main_df['month_enc'].unique()

array([0, 1, 2, 6, 5, 7, 3, 4])

In [43]:
def add_months(sourcedate, months):
    month = sourcedate.month - 1 + months
    year = sourcedate.year + month // 12
    month = month % 12 + 1
    day = min(sourcedate.day, calendar.monthrange(year,month)[1])
    return datetime.date(year, month, day)

add_months(reference_date, 1)

datetime.date(1997, 10, 20)

In [44]:
month_mapping = {i: add_months(reference_date, i) for i in main_df['month_enc'].values}
month_mapping

{0: datetime.date(1997, 9, 20),
 1: datetime.date(1997, 10, 20),
 2: datetime.date(1997, 11, 20),
 6: datetime.date(1998, 3, 20),
 5: datetime.date(1998, 2, 20),
 7: datetime.date(1998, 4, 20),
 3: datetime.date(1997, 12, 20),
 4: datetime.date(1998, 1, 20)}

In [15]:
def getPopularItems(df):
    df = df.copy()
    item_pop = df['item_id'].value_counts(ascending=False).to_dict()
    # srt_pop = {k: v for k, v in sorted(item_pop.items(), key=lambda item: item[0])}
    return item_pop

In [17]:
pop_items = getPopularItems(train_df)
list(zip(pop_items.keys(), pop_items.values()))[:10]


[(50, 484),
 (181, 422),
 (258, 402),
 (100, 395),
 (294, 394),
 (288, 391),
 (286, 388),
 (1, 383),
 (121, 353),
 (300, 352)]

In [31]:
item_id = 50
item_df = main_df.loc[main_df['item_id']==item_id, :]
item_df = item_df.groupby('month_enc').agg({'user_id': 'nunique', 'item_id': 'count', 'rating': 'mean'}).reset_index()
item_df.columns = ['month_enc', '# users subscribed', '# usages', 'average rating']
item_df

,month_enc,# users subscribed,# usages,average rating
0,0,40,40,4.250000
1,1,58,58,4.431034
2,2,125,125,4.264000
3,3,62,62,4.403226
4,4,59,59,4.440678
5,5,42,42,4.333333
6,6,59,59,4.322034
7,7,39,39,4.564103


In [36]:
sample_df = main_df.copy()
sample_df = sample_df.groupby('month_enc').agg({'user_id': ['nunique', 'count'], 'item_id': 'nunique', 'rating': ['mean', 'std']}).reset_index()
sample_df.columns = ['month_enc', '# active users', '# interaction', '# active items', 'mean rating', 'std rating']
sample_df

,month_enc,# active users,# interaction,# active items,mean rating,std rating
0,0,79,5077,959,3.526689,1.082641
1,1,152,8127,1082,3.581888,1.095785
2,2,286,19197,1283,3.563317,1.095851
3,3,181,9813,1208,3.580760,1.104195
4,4,217,11386,1311,3.378008,1.182847
5,5,169,8968,1289,3.455843,1.122204
6,6,224,10239,1219,3.549858,1.102143
7,7,158,7193,1247,3.601974,1.136624


In [45]:
sample_df['date'] = sample_df['month_enc'].map(month_mapping)
sample_df

,month_enc,# active users,# interaction,# active items,mean rating,std rating,date
0,0,79,5077,959,3.526689,1.082641,1997-09-20
1,1,152,8127,1082,3.581888,1.095785,1997-10-20
2,2,286,19197,1283,3.563317,1.095851,1997-11-20
3,3,181,9813,1208,3.580760,1.104195,1997-12-20
4,4,217,11386,1311,3.378008,1.182847,1998-01-20
5,5,169,8968,1289,3.455843,1.122204,1998-02-20
6,6,224,10239,1219,3.549858,1.102143,1998-03-20
7,7,158,7193,1247,3.601974,1.136624,1998-04-20


In [49]:
# generate monthly new user count
month_list = sample_df['month_enc'].values
all_users = [main_df.loc[main_df['month_enc']<=m, :]['user_id'].nunique() for m in month_list]
new_users = [all_users[i]-all_users[i-1] for i in range(1, len(all_users))]
new_users.insert(0, 0)
new_users

[0, 110, 237, 104, 117, 89, 133, 74]

In [50]:
sample_df['monthly new user engagement'] = new_users
sample_df

,month_enc,# active users,# interaction,# active items,mean rating,std rating,date,monthly new user engagement
0,0,79,5077,959,3.526689,1.082641,1997-09-20,0
1,1,152,8127,1082,3.581888,1.095785,1997-10-20,110
2,2,286,19197,1283,3.563317,1.095851,1997-11-20,237
3,3,181,9813,1208,3.580760,1.104195,1997-12-20,104
4,4,217,11386,1311,3.378008,1.182847,1998-01-20,117
5,5,169,8968,1289,3.455843,1.122204,1998-02-20,89
6,6,224,10239,1219,3.549858,1.102143,1998-03-20,133
7,7,158,7193,1247,3.601974,1.136624,1998-04-20,74
